In [1]:
# universally modules
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
sys.path.append("../src")

# preprocessing and transformation modules
import fasttext
import Preprocessing
from Features import buildFeatures
from Modelling import BaggingModelling
from Transformation import StackedTransformation

# Scikit-Learn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import TfidfVectorizer

# model algorithm
from xgboost import XGBRegressor, XGBClassifier

# evaluation modules
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report

/home/bigdata/anaconda3/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


### Parameters

In [2]:
# to speed up the process choose a sample size to randomly draw a sample of the whole daataset
sample_size = 1000 

# remove all text that contain less than n chars
min_chars_per_text = 50

# which features will be used for the TF-IDF transformation
text_features = "text_preprocessed"

#### define the target variable and categorial variables used in later transformations ###
algo_type = "regression" # regression or classification

#### Case 1: gender 
#target_variable = "gender"
#categorial_variables =  ["topic", "sign"]

# Case 2: topic
#target_variable = "topic"
#categorial_variables =  ["gender", "sign"]

# Case 3: age
target_variable = "age"
categorial_variables =  ["topic", "gender", "sign"]

# Case 4: sign
#target_variable = "sign"
#categorial_variables =  ["gender", "topic"]
############################################################################################

# use only words that occur at least sqrt_3(X) times 
min_df_exponent = (1/3)

### Initialization 

In [3]:
df = pd.read_csv("../resource/data/blogtext.csv")

# draw random sample for faster processing:
df = df.sample(sample_size)

### Filtering

In [4]:
# filter for a mininmal number of letters in a tweet:
df = df[df["text"].str.count(r"[a-zA-Z]") >= min_chars_per_text]
df = df.reset_index(drop=True)

### Feature Engineering

In [5]:
# append the data
features = [buildFeatures(text) for text  in tqdm(df["text"])]

# append the data
columns = ["Text length", "Number URLs", "Number mails",\
          "Uppercase ratio", "Lowercase ratio", "Number ratio", "Symbol ratio",\
          "Average letters per word", "Variance of letters per word", "Unique words ratio",\
          "Average letters per sentence", "Variance of letters per sentence",\
          "Average words per sentence", "Variance of words per sentence",\
          "Maximal uppercase ratio per sentence", "Length of the maximal uppercase ratio sentence"]

# merge the features with the original dataset
df_preprocessed = df.merge(pd.DataFrame(features, columns=columns), left_index=True, right_index=True)

100%|██████████| 907/907 [00:01<00:00, 716.74it/s]


### Text Preprocessing

In [6]:
# use the preprocessing  module
preprocessor = Preprocessing.Preprocessing()
df_preprocessed["text_preprocessed"] = preprocessor.ProcessMany(df_preprocessed["text"])

# predict the main language
model = fasttext.load_model('../src/data/lid.176.ftz')
df_preprocessed["main_language"] = [model.predict(text)[0][0].split("__")[-1] for text in tqdm(df_preprocessed["text_preprocessed"])]

100%|██████████| 907/907 [00:00<00:00, 6724.59it/s]


In [7]:
# drop unnecassary features
df_filtered = df_preprocessed[(df_preprocessed["main_language"] == "en")]\
                .drop(["id","text","date","main_language"], axis= 1)

### Data Split

In [8]:
X,y = df_filtered.drop(target_variable, axis=1),df_filtered[target_variable]

### Transformation

In [9]:
# use the  text transformer class to create two transformers for the textual and the numerical model
text_transformer = TfidfVectorizer(ngram_range=(1,1), min_df=int(len(X)**(min_df_exponent)))
numerical_transformer = make_column_transformer((OneHotEncoder(handle_unknown="ignore"), categorial_variables)\
                                                       , remainder=StandardScaler())

stacking = StackedTransformation(X, y, numerical_transformer, text_transformer, text_features)
stacking.build_transformer()

### Clustering

In [10]:
text_data_features = stacking.text_transformer.get_feature_names()
text_data = stacking.X_train_text_transformed.toarray()

df_text_cluster = pd.DataFrame(text_data, columns=text_data_features)

# Die Features beschreiben die Worte im Text
# Die Werte sind die TF*IDF-transformierten Textdaten

In [11]:
try:
    numerical_data = stacking.X_train_numerical_transformed.toarray()
except:
    numerical_data = stacking.X_train_numerical_transformed

numerical_data_features = np.append(stacking.numerical_transformer.transformers_[0][1].get_feature_names(),\
                     stacking.X_train_numerical.columns.drop(categorial_variables))

df_numerical_cluster = pd.DataFrame(numerical_data, columns=numerical_data_features)
#df_numerical_cluster


# Die Features mit den x0 - xi Werten beschreiben die Ausprägungen die kategorialen Variablen
# das jeweilige i beschreibt das i-te Element der im Punkt "Target Variable" definierten liste categorial_variables
# Die verbleibenden Features (ohne xi) sind Standardskaliert, (x - \mu)/\sigma

### Model Selection

### Training 

In [12]:
stacked_modelling = BaggingModelling(XGBRegressor, {}, XGBRegressor, {}, stacking, (0.5, 0.5))
X_test = stacked_modelling.stacked_transformation.X_test
y_test = stacked_modelling.stacked_transformation.y_test


stacked_modelling.fit()
stacked_modelling.create_report(X_test,y_test, algo_type)

Numerical model finished!
Text model finished!
Weights have been optimized:
                Textual model weight: 0.4812742269935398
                Numerical model weight: 0.5187257730064602


Absolute loss textual model              1095.045250
Absolute loss numerical model            1015.983943
Absolute loss equally weighted model      982.133860
Absolute loss optimized weights model     982.133860
dtype: float64

### Parameter Tuning 

### Model Evaluation 